In [1]:
from transformers import BertTokenizerFast, BertModel
tokenizer_bert = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model_bert = BertModel.from_pretrained("kykim/bert-kor-base")

inputs = tokenizer_bert("안녕하세요. 대한민국의 수도는 서울입니다. 캠핑장이 위치해있는 곳은 가평입니다.", return_tensors='pt')
output = model_bert(
    input_ids=inputs['input_ids'][:,:512],
    attention_mask=inputs['attention_mask'][:,:512],
    token_type_ids=inputs['token_type_ids'][:,:512]).pooler_output
print(inputs.__dict__)
print(output)

c:\Users\mnh51\anaconda3\envs\tour2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'data': {'input_ids': tensor([[    2, 16453,  2016, 23589, 14414,  8034, 14328, 13992,  2016, 33474,
          8018, 21457, 14140, 17045, 26827, 13992,  2016,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, '_encodings': [Encoding(num_tokens=18, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])], '_n_sequences': 1}
tensor([[-7.8625e-01,  5.2362e-01, -7.0118e-01,  8.0155e-01, -9.9779e-01,
         -8.3119e-02, -7.1623e-02, -5.5302e-01,  9.9877e-01,  8.1179e-01,
          1.6481e-02,  7.1726e-01,  3.6784e-01,  2.8335e-01,  7.5018e-01,
          9.4472e-01, -9.9866e-01,  1.4630e-01, -4.0840e-01,  9.8550e-01,
          2.2779e-01,  4.1843e-01,  8.9794e-01, -4.9380e-01, -6.2141e-01,
         -1.4762e-01, -6.9850e-01, -4.9428e-01, -8.0659e-02,  9.8356e-01,
         -6.4027e-01, -5.2773e-01, -3.7425e-01, -8.2617e-01, -

In [2]:
import pandas as pd
import torch

all_df = pd.read_csv('./train.csv')
print(type(all_df))
all_df['bert_emb'] = ""

i=0
for ov in all_df['overview']:
    inputs = tokenizer_bert(ov, return_tensors='pt')
    if len(inputs['input_ids'][0]) > 512:
        output = model_bert(
            input_ids=inputs['input_ids'][:,:512],
            attention_mask=inputs['attention_mask'][:,:512],
            token_type_ids=inputs['token_type_ids'][:,:512]
        ).pooler_output
    else:
        output = model_bert(
            input_ids=inputs['input_ids'][:,:512],
            attention_mask=inputs['attention_mask'][:,:512],
            token_type_ids=inputs['token_type_ids'][:,:512]
        ).pooler_output

    out = output.cpu().detach().numpy()
    all_df['bert_emb'][i] = out[0]
    i+=1

    if i%1000 == 0:
        print('i : ', i)

all_df.to_csv('./train_bert.csv')

<class 'pandas.core.frame.DataFrame'>


Token indices sequence length is longer than the specified maximum sequence length for this model (1125 > 512). Running this sequence through the model will result in indexing errors


i :  1000
i :  2000
i :  3000
i :  4000
i :  5000
i :  6000
i :  7000
i :  8000
i :  9000
i :  10000
i :  11000
i :  12000
i :  13000
i :  14000
i :  15000
i :  16000


In [3]:
import pandas as pd
import torch

from transformers import BertTokenizerFast, BertModel
tokenizer_bert = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model_bert = BertModel.from_pretrained("kykim/bert-kor-base")

all_df = pd.read_csv('./test.csv')
print(type(all_df))
all_df['bert_emb'] = ""

i=0
for ov in all_df['overview']:
    inputs = tokenizer_bert(ov, return_tensors='pt')
    if len(inputs['input_ids'][0]) > 512:
        output = model_bert(
            input_ids=inputs['input_ids'][:,:512],
            attention_mask=inputs['attention_mask'][:,:512],
            token_type_ids=inputs['token_type_ids'][:,:512]
        ).pooler_output
    else:
        output = model_bert(
            input_ids=inputs['input_ids'][:,:512],
            attention_mask=inputs['attention_mask'][:,:512],
            token_type_ids=inputs['token_type_ids'][:,:512]
        ).pooler_output

    out = output.cpu().detach().numpy()
    all_df['bert_emb'][i] = out[0]
    i+=1

all_df.to_csv('./test_bert.csv')